In [19]:
from ngsolve import *
from netgen.geom2d import unit_square
import numpy as np
import scipy.sparse as sparse


class ReducedBasis:

    def __init__(self):

        self.setSpace(np.pi, 0.4, 0.6)    
        self.setInterval(0, 5)

    def reset(self):
        
        self.V = None
        self.K_red = None
        self.M_red = None
        self.P_red = None
        self.sol = []
        self.norm_of_solutions_red = [] 
        self.norm_of_solutions_orig = [] # l2 norm 
        self.residual = []
        self.dim_orig= len(self.gfu.vec)# dim of original space
        self.dim_red  = None
        self.__snapshots_updated = True
        self.__snapshots = []
        self.indices = []
        self.__first_draw = True
        self.__drawmode = 'default'
        
    def setDrawmode(self, mode):
        allowed_modes = ['webgui', 'default']
        
        if mode in allowed_modes:
            self.__drawmode = mode
        else:
            self.__drawmode = 'default'
        self.__first_draw = True
        

    def setSpace(self, a, x_0, y_0, rb='dirichlet'):
        ## generate mesh
        geo = netgen.geom2d.SplineGeometry()

        p1 = geo.AppendPoint (0,0)
        p2 = geo.AppendPoint (a,0)
        p3 = geo.AppendPoint (a,a)
        p4 = geo.AppendPoint (0,a)

        geo.Append (["line", p1, p2], bc = "bottom")
        geo.Append (["line", p2, p3], bc = "right")
        geo.Append (["line", p3, p4], bc = "top")
        geo.Append (["line", p4, p1], bc = "left")

        self.mesh = Mesh(geo.GenerateMesh(maxh=0.1))

        if rb == 'dirichlet':
            self.fes = H1(self.mesh, order=5, dirichlet='top|bottom|left|right')
        else:
            self.fes = H1(self.mesh, order=5)
        u,v =self.fes.TnT()

        factor = 25

        func = exp(-factor*((x-x_0)**2 + (y-y_0)**2))

        self.f = LinearForm(self.fes)
        self.f += func * v * dx
        self.f.Assemble()

        self.omega = Parameter(0)
        self.a = BilinearForm(self.fes)
        self.a += (grad(u)*grad(v) - self.omega*self.omega *u*v) * dx
        self.a.Assemble()

        self.gfu = GridFunction(self.fes)
        self.drawu = GridFunction(self.fes)
        

        self.k_blf = BilinearForm(self.fes)
        self.k_blf += grad(u)*grad(v)*dx
        self.k_blf.Assemble()
        rows,cols,vals = self.k_blf.mat.COO()
        self.K_orig = sparse.csr_matrix((vals,(rows,cols)))

        self.m_blf = BilinearForm(self.fes)
        self.m_blf += u*v * dx
        self.m_blf.Assemble()
        rows,cols,vals = self.m_blf.mat.COO()
        self.M_orig = sparse.csr_matrix((vals,(rows,cols)))

        self.reset()


    def setInterval(self, beginning, end):
        self.beginning = beginning
        self.end = end

        self.reset()
        
    def save(self, out_dir):
        
        print("save in folder {}", out_dir)
        
        changeable = [self.V, self.K_red, self.M_red, self.P_red, self.sol, self.norm_of_solutions_red,                               self.norm_of_solutions_orig, self.residual, self.beginning, self.end, self.dim_orig, self.dim_red, 
         self.__snapshots, self.__snapshots_updated, self.indices]

        for j in range(len(changeable)):
            np.save(out_dir+str(j), changeable[j], allow_pickle=True)
            
    def load(self, in_dir):
        
        print("load from folder {}", in_dir)

        ending = ".npy"
        self.V = np.load(in_dir+str(1)+ending, allow_pickle=True)
        self.K_red = np.load(in_dir+str(2)+ending, allow_pickle=True)
        self.M_red = np.load(in_dir+str(3)+ending, allow_pickle=True)
        self.P_red = np.load(in_dir+str(4)+ending, allow_pickle=True)
        self.sol = np.load(in_dir+str(5)+ending).tolist()
        self.norm_of_solutions_red = np.load(in_dir+str(6)+ending).tolist()
        self.norm_of_solutions_orig = np.load(in_dir+str(7)+ending).tolist()
        self.residual = np.load(in_dir+str(8)+ending).tolist()
        self.beginning = np.load(in_dir+str(9)+ending)
        self.end = np.load(in_dir+str(10)+ending)
        self.dim_orig= np.load(in_dir+str(11)+ending)
        self.dim_red  = np.load(in_dir+str(12)+ending)
        self.__snapshots = np.load(in_dir+str(13)+ending)
        self.__snapshots_updated = np.load(in_dir+str(14)+ending)
        self.indices = np.load(in_dir+str(15)+ending).tolist()
        
        
    def setSnapshots(self, new_snapshots, reset = False):
        ## TODO: check format of snapshots
        if len(self.__snapshots) > 0 and not reset:
            print("append snapshots with {}".format(new_snapshots))
            self.__snapshots = np.append(self.__snapshots, np.array(new_snapshots))
        else:
            print("set snapshots and reset basis")
            self.__snapshots = np.array(new_snapshots)
            self.V = None
        self.__snapshots_updated = True
        tmp = self.__snapshots
        zip_to_sort = list(zip(tmp, range(len(tmp))))
        sorted_zip = sorted(zip_to_sort, key=lambda x: x[0], reverse=False)
        self.indices = [tup[1] for tup in sorted_zip]
    
    def getSnapshots(self, sorted = True):
        return self.__snapshots[self.indices]
          
                
    def __computeRB(self):

        print("compute Reduced Basis")


        if len(self.__snapshots) == 0:
            print(""" no snapshots given, please call 'instance.setSnapshots' first""")
            return
        
        _visual = False
        if _visual:
            Draw(self.gfu)

        self.dim_red = len(self.__snapshots)
        V_tmp = np.zeros((self.dim_orig, self.dim_red))

        try: 
            existing_basis_len = self.V.shape[1]
            V_tmp[:,0:existing_basis_len] = self.V
            print("extending basis")
        except:
            existing_basis_len = 0


        for n in range(0+existing_basis_len, self.dim_red):
            _omega = self.__snapshots[n]
            ## compute FEM solution for parameter _omega
            self.omega.Set(_omega)
            self.a.Assemble()
            self.gfu.vec.data = self.a.mat.Inverse(self.fes.FreeDofs(), inverse="sparsecholesky") * self.f.vec
            nof = Integrate(self.gfu*self.gfu, self.mesh)
            self.norm_of_solutions_orig += [nof]
            V_tmp[:,n] = np.array(self.gfu.vec.data) 
            self.sol += [self.gfu.vec]

            if _visual:
                Redraw()
                input("norm of solution {}: {}".format(_omega, nof))
                
        try:
            q, r = np.linalg.qr(V_tmp)
            self.V = V_tmp.dot(np.linalg.inv(r))
        except:
            print("Matrix is singular")

        self.K_red = np.transpose(self.V).dot(self.K_orig.dot(self.V))
        self.M_red = np.transpose(self.V).dot(self.M_orig.dot(self.V))
        self.P_red = np.transpose(self.V).dot(self.f.vec.data)
        
        self.__snapshots_updated = False
        print("finished computing Reduced Basis")


    def computeValues(self, param, residual=True, norm=True, *args, **kwargs):

        if self.__snapshots_updated:
            self.__computeRB()
        
        self.norm_of_solutions_red = []
        self.residual = []

        ured = GridFunction(self.fes)
        tmp = GridFunction(self.fes)

        j = 0
        norm_diff = []
        for _omega in param:
            if 'time' in args:
                t3 = Timer("t3")
                t3.Start()
            Ainv = np.linalg.inv(self.K_red-_omega*_omega*self.M_red)

            red_sol_vec = np.matmul(Ainv, self.P_red)

            ured.vec.FV().NumPy()[:] = self.V.dot(red_sol_vec)[:]
            if 'time' in args:
                t3.Stop()
            
            if norm:
                
                nof = Integrate(ured*ured, self.mesh)
                self.norm_of_solutions_red += [nof]
            
            if residual:
                ## TODO: efficient version 
                # print("compute Residual")
                
                ## TODO: A in sparse
                
                if 'numpy' in args:
                    if 'timer' in args:
                        t1 = Timer("t1")
                        t1a = Timer("t1a")
                        t1b = Timer("t1b")
                        t1.Start()
                        t1a.Start()
                    
                    tmp = self.K_orig.dot(np.array(ured.vec.data)
                        ) - _omega*_omega*self.M_orig.dot(np.array(ured.vec.data)
                        ) - self.f.vec.data

                    if 'timer' in args:
                        t1a.Stop()
                        t1b.Start()

                    res2 = np.linalg.norm(tmp[self.fes.FreeDofs()])

                    if 'timer' in args:
                        t1b.Stop()
                        t1.Stop()

                        t2.AddFlops(2*self.k_blf.mat.nze)
                        t2.Start()
                                
                
                ngs_temp = self.k_blf.mat.CreateColVector()
                ngs_temp.data = self.k_blf.mat*ured.vec - _omega*_omega*self.m_blf.mat*ured.vec - self.f.vec
                
                ngs_temp2 = self.k_blf.mat.CreateColVector()
                ngs_temp2.data = Projector(self.fes.FreeDofs(), True)*ngs_temp

                if 'timer' in args:
                    t2.Stop()
                
                res = Norm(ngs_temp2)
                
                self.residual += [res]

            j += 1


    def draw(self, omega):
        
        if self.__snapshots_updated:
            self.__computeRB()
        
        if self.__drawmode == 'webgui':
            from ngsolve.webgui import Draw

        elif self.__drawmode == 'default':
            import netgen.gui
            from ngsolve import Draw
            if self.__first_draw:
                Draw(self.drawu)
                self.__first_draw = False
        else: 
            print("unknown drawmode use instance.setDrawmode")
            return
            self.__first_draw = False

        ## TODO: solve lgs
        Ainv = np.linalg.inv(self.K_red-omega*omega*self.M_red)
        red_sol_vec = np.matmul(Ainv, self.P_red)
        
        self.drawu.vec.FV().NumPy()[:] = self.V.dot(red_sol_vec)[:]
        
        if self.__drawmode == 'webgui':
            if self.__first_draw:
                self.scene = Draw(self.drawu, self.mesh, websocket=True)
                self.__first_draw = False
            self.scene.Redraw()
        else:
            Redraw()

In [13]:
RBinst = ReducedBasis()
RBinst.setInterval(0,5)
params = np.arange(5)
RBinst.setSnapshots(params, reset=True)
random_omegas = np.sort(np.append(np.random.uniform(RBinst.beginning, RBinst.end, 50), params))

with TaskManager(int(1e8)):
    t2 = Timer("t2")
    RBinst.computeValues(random_omegas, norm=False)



set snapshots and reset basis
compute Reduced Basis
finished computing Reduced Basis


In [14]:
for t in Timers():
    print(t)

{'name': 'BaseVector::L2Norm', 'time': 0.0008662342877155183, 'counts': 55, 'flops': 1570855.0, 'Gflop/s': 1.8134297178916192}
{'name': 'Projector::Project', 'time': 0.001841901473807022, 'counts': 55, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'BaseVector::Add', 'time': 0.0026143451010967374, 'counts': 55, 'flops': 1570855.0, 'Gflop/s': 0.6008598479753169}
{'name': 'ScaleMatrix::MultAdd', 'time': 0.028019749479090732, 'counts': 55, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'SparseMatrix::MultAdd', 'time': 0.05914853088936493, 'counts': 110, 'flops': 96571310.0, 'Gflop/s': 1.6326916078546898}
{'name': 'gfu', 'time': 0.0, 'counts': 0, 'flops': 0.0, 'Gflop/s': nan}
{'name': 'gfu', 'time': 0.0, 'counts': 0, 'flops': 0.0, 'Gflop/s': nan}
{'name': 'Integrate CF', 'time': 0.0029351549745893855, 'counts': 5, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'SparseCholesky<d,d,d>::MultAdd fac2', 'time': 0.004533238286163277, 'counts': 5, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'SparseCholesky<d,d,d>::MultAdd fa

In [15]:
help(TaskManager)
## HI

Help on class TaskManager in module pyngcore:

class TaskManager(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      TaskManager
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(...)
 |      __enter__(self: pyngcore.TaskManager) -> None
 |  
 |  __exit__(...)
 |      __exit__(self: pyngcore.TaskManager, arg0: object, arg1: object, arg2: object) -> None
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: pyngcore.TaskManager) -> None
 |      
 |      2. __init__(self: pyngcore.TaskManager, pajetrace: int) -> None
 |      
 |      Run paje-tracer, specify buffersize in bytes
 |  
 |  __timing__(...)
 |      __timing__() -> std::__cxx11::list<std::tuple<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, double>, std::allocator<std::tuple<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocat

In [21]:
RBinst = ReducedBasis()
RBinst.setInterval(0,5)
params = np.arange(5)
RBinst.setSnapshots(params, reset=True)
random_omegas = np.sort(np.append(np.random.uniform(RBinst.beginning, RBinst.end, 200), params))

import cProfile, pstats, io

cp = cProfile.Profile()
cp.enable()

RBinst.computeValues(random_omegas)
cp.disable()

s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(cp, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())


set snapshots and reset basis
compute Reduced Basis
finished computing Reduced Basis
         5187 function calls (5185 primitive calls) in 2.835 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.835    1.417 /home/ams/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3293(run_code)
        2    0.000    0.000    2.835    1.417 {built-in method builtins.exec}
        1    0.000    0.000    2.835    2.835 <ipython-input-21-0f349f282590>:12(<module>)
        1    0.980    0.980    2.835    2.835 <ipython-input-19-7d8ba0996dfc>:206(computeValues)
        1    0.680    0.680    0.952    0.952 <ipython-input-19-7d8ba0996dfc>:153(__computeRB)
      210    0.719    0.003    0.719    0.003 {built-in method ngsolve.comp.Integrate}
      209    0.213    0.001    0.213    0.001 {method 'dot' of 'numpy.ndarray' objects}
      213    0.154    0.001    0.154    0.001 {built-in method num

In [24]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


RBinst = ReducedBasis()
RBinst.setInterval(0,5)
params = np.array(range(20))
RBinst.setSnapshots(params)

RBinst.setDrawmode("webgui")

# RBinst.draw(3)
f = lambda x: RBinst.draw(x)
interact(f, x=widgets.FloatSlider(min=RBinst.beginning, max=RBinst.end, step=0.05, value=3))



set snapshots and reset basis
compute Reduced Basis
finished computing Reduced Basis


<IPython.core.display.Javascript object>

NGSWebGuiWidget(render_data={'ngsolve_version': '6.2.2004-92-gb58c18ee', 'mesh_dim': 2, 'order2d': 2, 'order3d…

In [8]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()                      
ax.plot(random_omegas, newRBinst.norm_of_solutions_red, '-', label = "norm")

#ax.plot(newRBinst.getSnapshots(), residuals_snapshots[i][newindices], '.r', label = "residual snapshot")
ax.set_xlabel("omega")
ax.set_ylabel('norm')
plt.semilogy()
ax.legend()

NameError: name 'random_omegas' is not defined

In [9]:
import matplotlib.pyplot as plt
import os

fig, ax = plt.subplots()
out_dir = os.getcwd()+"/output/"
try:
    os.mkdir(path+out_dir)
except:
    print('no dir created')

RBinst = ReducedBasis()

beginning = RBinst.beginning
end= RBinst.end
eigenvalues = []
for j in range(beginning, end+1):
    for i in range(beginning, end+1):
        tmp = (i*i+j*j)**(1/2)
        if tmp <= end: eigenvalues += [tmp]
# print(eigenvalues)

stepsize = 0.5
first_snapshot = np.arange(RBinst.beginning,RBinst.end, stepsize)
MAX_IT = 10
snapshots = [None]*MAX_IT
norms_snapshots = [None]*MAX_IT
norms_red = [None]*MAX_IT
residuals = [None]*MAX_IT
residuals_snapshots = [None]*MAX_IT
random_omegas = np.sort(np.append(np.random.uniform(RBinst.beginning, RBinst.end, 1000), first_snapshot))

snapshots[0] = np.array(first_snapshot)
RBinst.setSnapshots(first_snapshot, reset = True)
# indices = range(len(snapshots[0]))
for i in range(0,MAX_IT):
    print("i = {}".format(i))

    RBinst.computeValues(snapshots[i])
    norms_snapshots[i] = np.array(RBinst.norm_of_solutions_orig)
    residuals_snapshots[i]= np.array(RBinst.residual)
    
    RBinst.computeValues(random_omegas)
    norms_red[i] = np.array(RBinst.norm_of_solutions_red)
    residuals[i] = np.array(RBinst.residual)
    
    ## plot and save
    plt.cla()
    ax.plot(random_omegas, residuals[i], '-', label = "residual")
    ax.plot(snapshots[i][RBinst.indices], residuals_snapshots[i][RBinst.indices], '.r', label = "residual snapshot")
    ax.set_xlabel("omega")
    ax.set_ylabel('residual')
    ax.set_title("residual {}".format(i))
    plt.semilogy()
    ax.legend()
    plt.savefig(out_dir+"residual {}".format(i))
#     plt.plot()

    plt.cla()
    ax.plot(random_omegas, norms_red[i], '-', label = "norm")
    ax.plot(eigenvalues, np.ones(len(eigenvalues)), '.', label="eigenvalues")
    ax.set_xlabel("omega")
    ax.set_ylabel('norm of reduced solution')
    ax.set_title("norm reduced solution {}".format(i))
    plt.semilogy()
    # ax.legend()
    plt.savefig(out_dir+"norm_red {}".format(i))
#     plt.plot()

    plt.cla()
    ax.plot(snapshots[i][RBinst.indices], norms_snapshots[i][RBinst.indices], '-', label = "norm")
    ax.set_xlabel("omega")
    ax.set_ylabel('norm of snapshot solution')
    ax.set_title("norm snapshot solution {}".format(i))
    plt.semilogy()
    # ax.legend()
    plt.savefig(out_dir+"norm_snap {}".format(i))
#     plt.plot()
    

    if i < MAX_IT-1:
        
        ## sort by residual
        tmp = residuals[i]
        zip_to_sort = list(zip(tmp, range(len(tmp))))
        sorted_zip = sorted(zip_to_sort, key=lambda x: x[0], reverse=True)
        tmp_ind = [tup[1] for tup in sorted_zip]
        for j in range(len(RBinst.indices)):
            if not (random_omegas[tmp_ind[j]] in snapshots[i]):
                max_omega = random_omegas[tmp_ind[j]]
                snapshots[i+1] = np.append(snapshots[i], max_omega)
                RBinst.setSnapshots(max_omega)
                break

        # tmp = snapshots[i+1]
        # zip_to_sort = list(zip(tmp, range(len(tmp))))
        # sorted_zip = sorted(zip_to_sort, key=lambda x: x[0], reverse=False)
        # indices = [tup[1] for tup in sorted_zip]

print('finished')


no dir created


NameError: name 'ReducedBasis' is not defined

In [10]:
RBinst.save("./saved_RBinst_/")

NameError: name 'RBinst' is not defined

In [11]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()                      
ax.plot(random_omegas, newRBinst.norm_of_solutions_red, '-', label = "norm")

# ax.plot(snapshots[i][indices], residuals_snapshots[i][indices], '.r', label = "residual snapshot")
ax.set_xlabel("omega")
ax.set_ylabel('norm')
plt.semilogy()
ax.legend()


NameError: name 'random_omegas' is not defined